In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import shapely.geometry
from numpy.typing import NDArray

import h5py
import folium
import shapely
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import imageio.v3 as iio
import matplotlib.pyplot as plt
from tqdm import tqdm

from pathlib import Path
from geovision.data.fmow import FMoW

In [ ]:
def load_image(df: pd.DataFrame, idx: int):
    return iio.imread(FMoW.local_staging/"rgb"/df.iloc[idx]["image_path"])

def plot_original_image_with_bboxes(df: pd.DataFrame, idx: int):
    row = df.iloc[idx]
    _, ax = plt.subplots(1,1, figsize = (10, 10))
    ax.imshow(load_image(df, idx))
    ax.add_patch(FMoW.get_rectangle_from_corners(
        (row["bbox_tl_0"], row["bbox_tl_1"]), (row["bbox_br_0"], row["bbox_br_1"]), fill = False, linewidth = 3, color = "red")
    )
    ax.add_patch(FMoW.get_rectangle_from_corners(
        (row["outer_bbox_tl_0"], row["outer_bbox_tl_1"]), (row["outer_bbox_br_0"], row["outer_bbox_br_1"]), fill = False, linewidth = 3, color = "blue")
    )
    
def crop_and_plot_image_with_bboxes(df: pd.DataFrame, idx: int):
    row = df.iloc[idx]
    image = load_image(df, idx) 
    image = image[row["outer_bbox_tl_0"]:row["outer_bbox_br_0"], row["outer_bbox_tl_1"]:row["outer_bbox_br_1"], :]

    _, ax = plt.subplots(1,1, figsize = (5,5))
    ax.imshow(image)
    ax.add_patch(FMoW.get_rectangle_from_corners(
        (row["inner_bbox_tl_0"], row["inner_bbox_tl_1"]), (row["inner_bbox_br_0"], row["inner_bbox_br_1"]), fill = False, linewidth = 3, color = "red")
    )

def plot_original_image_on_map(row: pd.Series):
    pass

def targer_bbox_to_geometry(row: pd.Series) -> shapely.geometry.Polygon:
    pass

def geometry_to_outer_bbox(row: pd.Series) -> tuple[int, int, int, int]:
    pass

def calculate_outer_bbox_from_geometry(row: pd.Series) -> tuple[int, int, int, int]:
    # long, lat <==> width, height
    pixel_dims = row["mean_pixel_width"], row["mean_pixel_height"]
    outer_tl = np.array(row["geometry"].exterior.coords[0])
    inner_tl, _, inner_br, _ = [np.array(point) for point in row["intersect"].exterior.coords[:4]]

    tl = np.floor((inner_tl - outer_tl) * pixel_dims).astype(np.uint32)
    br = np.ceil((inner_br - outer_tl) * pixel_dims).astype(np.uint32)

    return tl[1], tl[0], br[1], br[0]

def bbox_to_geometry(geometry, height, width, tl_0, tl_1, br_0, br_1, step_0, step_1) -> shapely.geometry.Polygon:
    # geometry conventions are long, lat because of epsg:4326, opposite of bbox row, column
    tl, tr, br, bl = [np.array(point) for point in geometry.exterior.coords[:4]]

    def shift_right(point: NDArray, by: int):
        point[0] += by*step_1
        return point

    def shift_down(point: NDArray, by: int):
        point[1] += by*step_0
        return point 

    tl = shift_down(tl, tl_0)
    tl = shift_right(tl, tl_1)

    tr = shift_down(tr, tl_0)
    tr = shift_right(tr, br_1 - width)

    br = shift_down(br, br_0 - height)
    br = shift_right(br, br_1 - width)

    bl = shift_down(bl, br_0 - height)
    bl = shift_right(bl, tl_1)

    return shapely.Polygon([tl, tr, br, bl, tl])

In [ ]:
df = FMoW.get_multiclass_classification_df_from_metadata()
df = df[df.country_code == "IND"].reset_index()

In [ ]:
plot_original_image_with_bboxes(df, 0)

In [ ]:
r = df.iloc[10]
print(r.utm)
r.geometry.project(r.utm)

In [ ]:
row = df.iloc[0]
print(row.mean_pixel_height, row.mean_pixel_width)
tl_0, tl_1, br_0, br_1 = row.geometry.bounds

bbox_tl_0, bbox_tl_1, bbox_br_0, bbox_br_1 = bbox_to_geometry(
    row.geometry, row.img_height, row.img_width, 
    row.bbox_tl_0, row.bbox_tl_1, row.bbox_br_0, row.bbox_br_1, 
    row.mean_pixel_height, row.mean_pixel_width
).bounds

outer_bbox_tl_0, outer_bbox_tl_1, outer_bbox_br_0, outer_bbox_br_1 = bbox_to_geometry(
    row.geometry, row.img_height, row.img_width, 
    row.outer_bbox_tl_0, row.outer_bbox_tl_1, row.outer_bbox_br_0, row.outer_bbox_br_1, 
    row.mean_pixel_height, row.mean_pixel_width
).bounds

print(row.label_str, row.country_code, row.geometry.centroid.coords[0])

map = folium.Map(location = tuple(reversed(row.geometry.centroid.coords[0])))
folium.TileLayer(tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(map)

map.add_child(folium.Rectangle(bounds = ((tl_1, tl_0), (br_1, br_0))))
map.add_child(folium.Rectangle(bounds = ((bbox_tl_1, bbox_tl_0), (bbox_br_1, bbox_br_0)), color = "red"))
map.add_child(folium.Rectangle(bounds = ((outer_bbox_tl_1, outer_bbox_tl_0), (outer_bbox_br_1, outer_bbox_br_0))))

In [ ]:
idx = 350840 
wkt_str = str(df.iloc[idx]["raw_location"]).removeprefix("POLYGON ((").removesuffix("))")
polygon = shapely.from_wkt(df.iloc[idx]["raw_location"])

In [ ]:
plt

In [ ]:
# Image Height vs Width
# Insight: very few images are larger than 2000x2000px 
sns.jointplot(df, x = "width", y = "height")
print(df.apply(lambda x: True if x["width"] > 2000 or x["height"] > 2000 else False, axis = 1).sum(), "images have a dimention larger than 2000x2000px")

In [ ]:
# Classwise Distribution of Heights and Widths
# Insight: airport, amusement_park, impoverished_settlement, nuclear_powerplant, port, runway, shipyard and space facility are
# the ones with images > 2000px mostly, with a few outliers from each class
fig, axes = plt.subplots(1, 2, figsize = (20, 15))
sns.boxplot(df, y = "label_str", x = "height", ax = axes[0])
sns.boxplot(df, y = "label_str", x = "width", ax = axes[1])

In [ ]:
sns.jointplot(df, x = "bbox_width", y = "bbox_height")
print(df.apply(lambda x: True if x["width"] > 2000 or x["height"] > 2000 else False, axis = 1).sum(), "bboxes > 2000x2000px")

In [ ]:
df["bbox_height"] = df.apply(lambda x: x["bbox_br_0"] - x["bbox_tl_0"], axis = 1) 
df["bbox_width"] = df.apply(lambda x: x["bbox_br_1"] - x["bbox_tl_1"], axis = 1) 

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (20, 15))
sns.boxplot(df, y = "label_str", x = "bbox_height", ax = axes[0])
sns.boxplot(df, y = "label_str", x = "bbox_width", ax = axes[1])

In [ ]:
# Classwise distribution of images and pixels enclosed by bboxes
fig, axes = plt.subplots(1,2, figsize = (20, 15))
df["num_bbox_pixels"] = df.apply(lambda x: x["bbox_height"]*x["bbox_width"], axis = 1)
sns.barplot(df.groupby("label_str").count().reset_index(drop = False), y = "label_str", x = "image_path", ax = axes[0])
sns.barplot(df[["label_str", "num_bbox_pixels"]].groupby("label_str").sum().reset_index(drop = False), y = "label_str", x = "num_bbox_pixels", ax = axes[1])